In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
url = 'https://es.wikipedia.org/wiki/Anexo:Tabla_hist%C3%B3rica_de_la_Copa_Libertadores'
border = '1'

In [ ]:
r = requests.get(url)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
df = soup.find('table', attrs = {'border':border})

In [ ]:
table = pd.read_html(str(df))

In [ ]:
copa_libertadores = pd.DataFrame(table[0])

In [ ]:
copa_libertadores.head(10)

In [ ]:
tr = str.maketrans('áéíóú', 'aeiou')

In [ ]:
# modificando las columnas
copa_libertadores.columns = copa_libertadores.columns.str.lower().str.replace('.','').str.translate(tr)
copa_libertadores.rename(columns = {'pos': 'posicion', 'part': 'participaciones'}, inplace = True)
copa_libertadores.columns

In [ ]:
# arreglando la serie de diferencia de goles (dg)
copa_libertadores.dg = [value.strip('+').replace('–', '-') for value in copa_libertadores.dg]

In [ ]:
copa_libertadores.info()

In [ ]:
# arreglando el tipo de datos
copa_libertadores.pais = copa_libertadores.pais.astype('category')
copa_libertadores.dg = copa_libertadores.dg.astype(int)

# Estadisticas de equipos de peruanos en la Copa Libertadores

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import numpy as np

In [ ]:
copa_libertadores.pais.value_counts()

In [ ]:
# seleccionamos los 23 equipos peruanos que han participado en la copa libertadores
eq_per = copa_libertadores[copa_libertadores.pais == 'Perú']
eq_per.head(10)

In [ ]:
sns.set()

In [ ]:
# número de ediciones, victorias totales
fig, ax = plt.subplots(1, 2)
fig.set_size_inches([20,5])

ax[0].bar(eq_per.equipo, eq_per.participaciones, color ='c')
ax[0].set_xlabel('Equipo', fontsize = 13, fontweight = 'bold')
ax[0].set_ylabel('Número de participaciones', fontsize = 13, fontweight = 'bold')
ax[0].set_title('PARTICIPACIONES POR EQUIPO', fontsize = 'x-large', fontweight = 'bold')
ax[0].set_xticklabels(eq_per.equipo, rotation = 90)
ax[0].tick_params(labelsize = 13)

ax[1].bar(eq_per.equipo, eq_per.pg, color = 'm')
ax[1].set_xlabel('Equipo', fontsize = 13, fontweight = 'bold')
ax[1].set_ylabel('Victorias totales conseguidas', fontsize = 13, fontweight = 'bold')
ax[1].set_title('VICTORIAS POR EQUIPO', fontsize = 'x-large', fontweight = 'bold')
ax[1].set_xticklabels(eq_per.equipo, rotation = 90)
ax[1].tick_params(labelsize = 13)
plt.show()

In [ ]:
# goles totales, puntos totales
fig, ax = plt.subplots(1,2)
fig.set_size_inches([20,5])

ax[0].bar(eq_per.equipo, eq_per.gf, color ='b')
ax[0].set_xlabel('Equipo', fontsize = 13, fontweight = 'bold')
ax[0].set_ylabel('Cantidad de Goles Totales', fontsize = 13, fontweight = 'bold')
ax[0].set_title('GOLES TOTALES POR EQUIPO', fontsize = 'x-large', fontweight = 'bold')
ax[0].set_xticklabels(eq_per.equipo, rotation = 90)
ax[0].tick_params(labelsize = 13)

ax[1].bar(eq_per.equipo, eq_per.ptos, color ='g')
ax[1].set_xlabel('Equipo', fontsize = 13, fontweight = 'bold')
ax[1].set_ylabel('Cantidad de Puntos Totales', fontsize = 13, fontweight = 'bold')
ax[1].set_title('PUNTOS TOTALES POR EQUIPO', fontsize = 'x-large', fontweight = 'bold')
ax[1].set_xticklabels(eq_per.equipo, rotation = 90)
ax[1].tick_params(labelsize = 13)

In [ ]:
def ecdf(data):
    """Cálculo de la funcón de distribución acumulada empírica."""
    # Número de observaciones: n
    n = len(data)
    # x-valores para la ECDF: x
    x = sorted(data)
    # y-valores para la ECDF: y
    y = np.arange(1, n+1) / n
    return x, y

In [ ]:
# cumulative distribution function (puntaje)
x_puntos, y_puntos = ecdf(copa_libertadores.ptos)
plt.figure(figsize=(8,5))
plt.plot(x_puntos, y_puntos, marker = '.', linestyle = 'none')
plt.ylabel('CDF')
plt.xlabel('Puntos Totales en la Copa')
plt.annotate('Universitario', xy = (220,0.905213), xytext = (400,0.8), 
             arrowprops = dict(facecolor='black', shrink = 0.05), fontsize = 15)
plt.show()

In [ ]:
# identificar los percentiles: u, sc y al
cl_ecdf = pd.DataFrame({'x':x_puntos, 'y':y_puntos})
universitario = np.round((1 - cl_ecdf[cl_ecdf.x == 220]['y'])*100, 2)
cristal = np.round((1 - cl_ecdf[cl_ecdf.x == 184]['y'])*100, 2)
alianza = np.round((1 - cl_ecdf[cl_ecdf.x == 142]['y'])*100, 2)
print('PERCENTILES \n')
print('UNIVERSITARIO: ' + str(universitario.to_string(index = False)) +'%\n'\
     'SPORTING CRISTAL: ' + str(cristal.to_string(index = False)) +'%\n'\
     'ALIANZA LIMA: ' + str(alianza.to_string(index = False)) +'%')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# victorias/edición, goles/edición, puntos/edición, victorias/partidos, goles/partidos, puntos/partidos
# indicadores por edición
eq_per['pg/ed'] = np.round(eq_per['pg']/eq_per['participaciones'], 2)
eq_per['gf/ed'] = np.round(eq_per['gf']/eq_per['participaciones'], 2)
eq_per['pts/ed'] = np.round(eq_per['ptos']/eq_per['participaciones'], 2)
# indicadores por partido
eq_per['pg/pj'] = np.round(eq_per['pg']/eq_per['pj'], 2)
eq_per['pe/pj'] = np.round(eq_per['pe']/eq_per['pj'], 2)
eq_per['pp/pj'] = np.round(eq_per['pp']/eq_per['pj'], 2)
eq_per['gf/pj'] = np.round(eq_per['gf']/eq_per['pj'], 2)
eq_per['pts/pj'] = np.round(eq_per['ptos']/eq_per['pj'], 2)

In [ ]:
# selección de los 10 mejores equipos peruanos que participaron de la Copa Libertadores
peru_10 = eq_per.head(10)
peru_10

In [ ]:
# pg/ed, gf/ed, pts/ed
n = len(peru_10)
ind = np.arange(n)
width = 0.25
fig, ax = plt.subplots()
fig.set_size_inches([15,5])
cont1 = ax.bar(ind, peru_10['pg/ed'], width, color='b')
cont2 = ax.bar(ind+width, peru_10['gf/ed'], width, color='g')
cont3 = ax.bar(ind+width*2, peru_10['pts/ed'], width, color='m')
ax.set_ylabel('Ratios', fontsize = 13, fontweight = 'bold')
ax.set_xlabel('Equipos', fontsize = 13, fontweight = 'bold')
ax.set_title('VICTORIAS, GOLES Y PUNTOS PROMEDIO POR PARTICIPACIÓN', fontsize = 'x-large', fontweight = 'bold')
ax.set_xticks(ind + width)
ax.set_xticklabels((peru_10.equipo), rotation = 90)
ax.legend((cont1[0], cont2[0], cont3[0]), ('pg/ed', 'gf/ed', 'pts/ed'))
def autolabel(conts):
    for cont in conts:
        height = cont.get_height()
        ax.text(cont.get_x() + cont.get_width()/2., 1.01*height,
                '%.2f' % height,
                ha='center', va='bottom')
autolabel(cont1)
autolabel(cont2)
autolabel(cont3)
plt.show()

In [ ]:
# pg/pj, pe/pj, pp/pj
fig, ax = plt.subplots()
fig.set_size_inches([7,7])
ax.bar(peru_10.equipo, peru_10['pp/pj'], label = 'pp/pj')
ax.bar(peru_10.equipo, peru_10['pe/pj'], bottom = peru_10['pp/pj'], label = 'pe/pj')
ax.bar(peru_10.equipo, peru_10['pg/pj'], bottom = peru_10['pp/pj'] + peru_10['pe/pj'], label = 'pg/pj')
ax.set_xlabel('Equipo', fontweight = 'bold')
ax.set_ylabel('Ratios', fontweight = 'bold')
ax.set_title('PORCIÓN DE VICTORIAS, EMPATES Y DERROTAS', fontweight = 'bold')
ax.set_xticklabels(peru_10.equipo, rotation = 90)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# gf/pj, pts/pj
n = len(peru_10)
ind = np.arange(n)
width = 0.35
fig, ax = plt.subplots()
fig.set_size_inches([15,5])
cont11 = ax.bar(ind, peru_10['gf/pj'], width, color='r')
cont12 = ax.bar(ind+width, peru_10['pts/pj'], width, color='y')
ax.set_ylabel('Ratios', fontsize = 13, fontweight = 'bold')
ax.set_xlabel('Equipos', fontsize = 13, fontweight = 'bold')
ax.set_title('GOLES PROMEDIO POR PARTIDO Y PUNTOS PROMEDIO POR PARTIDO', fontsize = 'x-large', fontweight = 'bold')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels((peru_10.equipo), rotation = 90)
ax.legend((cont11[0], cont12[0]), ('gf/pj', 'pts/pj'))
autolabel(cont11)
autolabel(cont12)
plt.show()

In [ ]:
# los equipos de toda la copa con la peor diferencia de goles
copa_libertadores.nsmallest(5, 'dg')

In [ ]:
# ver si hay relación entre los partidos jugados y la diferencia de goles
fig, ax = plt.subplots(figsize = (15,5))
markers = ['o', 'v', '^', '<', '>', 'D', '8', 'P', 's', '*', 'X']
sns.scatterplot(x='pj', y='dg', data=copa_libertadores, hue='pais', style = 'pais', markers = markers, 
                palette=sns.hls_palette(11, l=.4), ax=ax)
ax.set_xlabel('Partidos Jugados', fontweight = 'bold')
ax.set_ylabel('Diferencia de Goles', fontweight = 'bold')
ax.set_title('RELACIÓN ENTRE PARTIDOS JUGADOS Y DIFERENCIA DE GOLES', fontweight = 'bold')
ax.legend(loc = 'center left', bbox_to_anchor = (1.05,0.5))
ax.axhline(color = 'k', alpha = 0.7, linestyle = '--')
ax.annotate('"Alianza Lima"', xy=(164, -169), xytext=(200, -150), arrowprops=dict(facecolor='black', shrink=0.05), fontsize=15)
plt.show()

# Estadísticas del fútbol peruano en la Copa Libertadores

In [ ]:
# estadísticas de los 10 mejores equipos peruanos
eq_per.iloc[:10, 4:13].describe()[1:]

In [ ]:
# los mayores exponentes por país de acuerdo al ranking o puntos
copa_libertadores.groupby('pais').first().sort_values('posicion')

In [ ]:
# mostrar estadísticas agregadas por país: media, suma, mediana
copa_libertadores.groupby('pais')[['participaciones', 'titulos', 'ptos']].agg([np.sum, np.mean, np.median])\
    .sort_values([('titulos','sum'),('ptos','sum')], ascending = False)

# THE END